# Train example notebook

This notebook is used to implement the training of a neural network for classification of `Cloud`, `Edge`, `Good` images. <br> It is advisable to use this notebook to get practice and debug your code. To speed up the execution, once you are ready, you should move to a scripted version.

## 1. - Imports

Select `CUDA_VISIBLE_DEVICES` to the `Graphics Proceesing Unit (GPU)` index that you want to use to enable the use of GPU.

In [114]:
import os 
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0" # GPU index
print(os.environ["CUDA_VISIBLE_DEVICES"])
print(torch.cuda.is_available())

import torch
import torch.nn as nn

# Check available GPUs
print("Available GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

0
True
Available GPUs: 1
GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU


Enabling autoreload of different packages.

In [115]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
import torch 
import sys
sys.path.insert(1, os.path.join("..", "data"))
sys.path.insert(1, os.path.join("..", "utils"))
from data_utils import Dataset
from plot_utils import plot_image
from torch.utils.data import DataLoader

## 2. - Datasets

### 2.1 - Creating datasets

Now we read the images from the target directory `path_data`. Set `path_data` to the directory containing the `Cloud`, `Edge`, `Good` subfolders.  Moreover, it will automatically split the total dataset into the train, cross validation and test splits by using a pseudo-random splitting algorithm. You can reproduce the split by specifying the variable `seed`. **NB**:
- The train split contains 70% of the whole images.
- The valid splits contains 15% of the whole images.
- The test splits contains 15% of the whole images.
<br>**YOU MUST NOT CHANGE THE TEST SPLIT SIZE!!!**

In [117]:
# Path to the data folder (update the variable to your path).
path_data=os.path.join("..", "data")
# Seed value
seed=15

<img src="utilities/images/danger_icon.png" style="margin:auto"/>

**N.B** Make sure to have created a dataset split into the three directories `Cloud`, and `Good`, `Edge`. Otherwise, the next cell will **fail!** <br>


In [118]:
dataset=Dataset(path_data=path_data, seed=seed)
dataset.read_data()

Parsing class: Cloud: 141it [00:07, 18.98it/s]
Parsing class: Edge: 97it [00:04, 21.50it/s]
Parsing class: Good: 66it [00:03, 18.66it/s]


**Hint:** before proceeding, make sure that your `Edge`,`Cloud`, and `Good` samples are well enough among the `train`, `valid`,`test` splits. To print datasets statistics, run the next line.  Remember that the number of images in the different splits is distributed as described above. <br> If you are not happy with the data distribution, you can update the seed used and create a new dataset by rerunning the cell above. 

In [119]:
dataset.get_statistics()

,train,valid,test
cloud,97,22,22
edge,70,15,12
good,45,9,12


### 2.2. - Create data loaders.

The next lines will create a dataloader. A data loader is used to break the dataset into batches of a size `batch_size`. <br> This is useful to ensure that your dataset will fit into your memory and to create a "stochastic" implementation of gradient descent. <br> For more information, please, check: [data loader](https://www.educative.io/answers/what-is-pytorch-dataloader).<br>
Specify `batch_size` (**Hint**: use powers of 2. Typical values are between 8 and 64).

In [120]:
batch_size=32

In [121]:
# Train loader
train_loader = DataLoader(dataset.get_split("train"), batch_size=batch_size, pin_memory=False, shuffle=True)
# Cross validation data loader
valid_loader = DataLoader(dataset.get_split("valid"), batch_size=46, pin_memory=False, shuffle=True)
# Test data loader
test_loader = DataLoader(dataset.get_split("test"), batch_size=batch_size, pin_memory=False, shuffle=True)

## 3 - Training

Now, it is your turn! Add your code below to load a Neural Network model, select optimizers, learning rate and perform training. <br>
Good luck!

In [122]:
classes = ('cloud', 'edge', 'good')

In [123]:
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid 
import torch.nn.functional as F 
import numpy as np
import matplotlib.pyplot as plt

def resize_tensor_images(images, size=(256, 256)):
    # Resize the batch of images
    return F.interpolate(images, size=size, mode='bilinear', align_corners=False)


def compute_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count

    return mean, std

def normalize_images(images, mean, std):
    normalized_images = (images - mean.view(-1, 1, 1)) / std.view(-1, 1, 1)
    return normalized_images
    
def tensor_to_numpy(tensor):
    # Rescale the tensor to 0-1 range
    tensor = tensor - tensor.min()
    tensor = tensor / tensor.max()
    # Move the tensor to CPU if it's on GPU
    tensor = tensor.cpu()

    # Convert to numpy and transpose from CxHxW to HxWxC for visualization
    numpy_image = tensor.numpy()
    numpy_image = np.transpose(numpy_image, (1, 2, 0))

    return numpy_image

def normalize_individual_image(image):
    # Calculate the mean and std for each channel of the image
    mean = image.mean(dim=[1, 2])
    std = image.std(dim=[1, 2])

    # Ensure std is not zero to avoid division by zero
    std = std.clamp(min=1e-9)

    # Normalize the image
    normalized_image = (image - mean[:, None, None]) / std[:, None, None]
    return normalized_image

normalized_batches_TRL = []
normalized_batches_VAL = []
normalized_batches_test = []
for batch in train_loader:
    images, labels = batch

    resized_images_TRL = resize_tensor_images(images)

    # Normalize each image in the batch
    normalized_images_TRL = torch.stack([normalize_individual_image(img) for img in resized_images_TRL])
    normalized_batches_TRL.append((normalized_images_TRL, labels))

for batch in valid_loader:
    images, labels = batch

    resized_images_VAL = resize_tensor_images(images)

    # Normalize each image in the batch
    normalized_images_VAL = torch.stack([normalize_individual_image(img) for img in resized_images_VAL])
    normalized_batches_VAL.append((normalized_images_VAL, labels))

normalized_alldata_batches_TRL = []
normalized_alldata_batches_VAL = []
normalized_alldata_batches_test = []
mean, std = compute_mean_std(train_loader)
for batch in train_loader:
    images, labels = batch

    resized_images_TRL = resize_tensor_images(images)
    # Normalize the batch of images
    normalized_alldata_images_TRL = normalize_images(resized_images_TRL, mean, std)

    # Append the normalized images and their corresponding labels to the list
    normalized_alldata_batches_TRL.append((normalized_alldata_images_TRL, labels))

mean, std = compute_mean_std(valid_loader)
for batch in valid_loader:
    images, labels = batch

    resized_images_VAL = resize_tensor_images(images)
    # Normalize the batch of images
    normalized_alldata_images_VAL = normalize_images(resized_images_VAL, mean, std)

    # Append the normalized images and their corresponding labels to the list
    normalized_alldata_batches_VAL.append((normalized_alldata_images_VAL, labels))

mean, std = compute_mean_std(test_loader)
for batch in test_loader:
    images, labels = batch

    resized_images_test = resize_tensor_images(images)
    # Normalize the batch of images
    normalized_alldata_images_test = normalize_images(resized_images_test, mean, std)

    # Append the normalized images and their corresponding labels to the list
    normalized_alldata_batches_test.append((normalized_alldata_images_test, labels))

non_normalized_batches_TRL = []
non_normalized_batches_VAL = []

for batch in train_loader:
    images, labels = batch

    resized_images_TRL = resize_tensor_images(images)
    non_normalized_TRL = resize_tensor_images(images) / 255
    non_normalized_batches_TRL.append((non_normalized_TRL, labels))

for batch in valid_loader:
    images, labels = batch

    resized_images_VAL = resize_tensor_images(images)
    non_normalized_VAL = resize_tensor_images(images) / 255
    non_normalized_batches_VAL.append((non_normalized_VAL, labels))


# first_image_tensor = normalized_alldata_images_TRL[0]

# # Convert the tensor to a NumPy array
# first_image_numpy = tensor_to_numpy(first_image_tensor)

# plt.imshow(first_image_numpy)
# plt.axis('off')  # Remove axis markers


In [124]:


import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)  # New convolutional layer
        self.fc1 = nn.Linear(32 * 32 * 32, 160)  # Adjusted for the new layer
        self.fc2 = nn.Linear(160, 100)
        self.fc3 = nn.Linear(100, 50)
        self.fc4 = nn.Linear(50, 3)  # New fully connected layer

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))  # Apply the new layer
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)  # Apply the new fully connected layer
        return x

net = Net()



In [125]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [126]:
for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(normalized_alldata_batches_TRL, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs)
        print(labels)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

tensor([0, 0, 2, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 1])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 1, 1,
        1, 0, 0, 0, 0, 1, 2, 2])
tensor([0, 0, 2, 0, 2, 0, 2, 2, 2, 1, 0, 1, 0, 2, 0, 1, 0, 2, 0, 1, 1, 0, 2, 0,
        0, 1, 1, 0, 0, 0, 1, 0])
tensor([2, 0, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 2,
        1, 2, 0, 1, 0, 1, 0, 1])
tensor([1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 2, 0, 0,
        0, 0, 1, 0, 2, 0, 2, 2])
tensor([1, 1, 1, 2, 0, 1, 2, 2, 0, 1, 0, 0, 2, 2, 0, 1, 0, 0, 1, 1, 2, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 1])
tensor([1, 0, 0, 2, 0, 2, 2, 1, 0, 2, 1, 2, 2, 0, 0, 2, 1, 1, 1, 0])
tensor([0, 0, 2, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 2, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 1])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 1, 1,
        1, 0, 0, 0, 0, 1, 2, 2])
tensor([0, 0, 2, 0, 2, 0, 2

In [ ]:
PATH = './test1.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(normalized_alldata_batches_VAL)
images, labels = next(dataiter)

dataiter_train = iter(normalized_alldata_batches_TRL)
images_train, labels_train = next(dataiter_train)

dataiter_test = iter(normalized_alldata_batches_test)
images_test, labels_test = next(dataiter_test)
# print images
# for i in range(len(images)):
#     first_image_tensor = images[i]

#     # Convert the tensor to a NumPy array
#     first_image_numpy = tensor_to_numpy(first_image_tensor)

#     # Display the image
#     plt.imshow(first_image_numpy)
#     plt.axis('off')  # Remove axis markers
#     plt.show()


In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
outputs = net(images)
outputs_train = net(images_train)
outputs_test = net(images_test)

print(outputs)

tensor([[  6.3717,  -4.3532,  -0.8758],
        [ -4.8686,   7.6854,  -6.5024],
        [  2.3771,  -1.7594,  -1.1256],
        [  2.3855,  -1.2246,  -0.8786],
        [  2.4407,  -0.9329,  -1.5980],
        [  0.5432,   0.5281,  -1.0848],
        [ -9.4273,  10.4462,  -5.2313],
        [ -7.5458,   7.4785,  -6.0561],
        [  1.0999,  -7.3880,   6.1811],
        [ -0.2073,   4.2922,  -4.3811],
        [ -1.8541,   2.7516,  -3.1666],
        [ -7.0683,   7.2468,  -3.7786],
        [  1.7555,  -0.9721,  -0.2960],
        [  3.0394,  -4.7818,   1.8282],
        [  1.9154,  -0.8557,  -0.9290],
        [  7.0310,  -7.2639,   0.7705],
        [ -1.6838,   7.1378,  -6.0082],
        [  0.6975,  -1.4552,   0.9193],
        [  6.6812,  -2.4990,  -4.3222],
        [  2.3911,  -1.4056,  -0.7675],
        [  1.7738,  -0.7072,  -0.8411],
        [  4.1534,  -1.7177,  -1.9793],
        [  1.9767,  -1.4979,  -0.1531],
        [ -5.8494,   5.2779,  -0.7431],
        [  2.2142,  -1.0921,  -1.0410],


In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(32)))
# Assume 'outputs' are the logits from your network and 'labels' are your ground truth labels

# Convert outputs to predicted class labels
_, predicted_train = torch.max(outputs_train, 1)
_, predicted_test = torch.max(outputs_test, 1)

# Calculate the number of correctly predicted labels
correct_predictions = (predicted == labels).sum().item()
correct_predictions_train = (predicted_train == labels_train).sum().item()
correct_predictions_test = (predicted_test == labels_test).sum().item()
# Calculate the total number of labels
total_labels = labels.size(0)
total_labels_train = labels_train.size(0)
total_labels_test = labels_test.size(0)
# Calculate the accuracy as a percentage
accuracy = 100 * correct_predictions / total_labels
accuracy_train = 100 * correct_predictions_train / total_labels_train
accuracy_test = 100 * correct_predictions_test / total_labels_test
print('Validation Accuracy: {:.2f}%'.format(accuracy))
print('Train accuracy: {:.2f}%'.format(accuracy_train))
print('Test accuracy: {:.2f}%'.format(accuracy_test))

Predicted:  cloud edge  cloud cloud cloud cloud edge  edge  good  edge  edge  edge  cloud cloud cloud cloud edge  good  cloud cloud cloud cloud cloud edge  cloud cloud cloud cloud edge  cloud cloud cloud
Validation Accuracy: 80.43%
Train accuracy: 84.38%
Test accuracy: 56.25%
